In [1]:
# you will "rewind" your graph by fetching a checkpoint using 
# the graph's get_state_history method. We can then resume 
# execution at this previous point in time.

from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from dotenv import load_dotenv
load_dotenv()

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tool = TavilySearchResults(max_results=2)
tools = [tool]
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [2]:
config = {"configurable": {"thread_id": "1"}}
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "I'm learning LangGraph. "
                    "Could you do some research on it for me?"
                )
            }
        ]
    },
    config,
    stream_mode="values"
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_kHZvVIExpbA02rn8LA56SDpk)
 Call ID: call_kHZvVIExpbA02rn8LA56SDpk
  Args:
    query: LangGraph tutorial documentation
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://langchain-ai.github.io/langgraph/tutorials/introduction/", "content": "[](https://langchain-ai.github.io/langgraph/tutorials/introduction/#__codelineno-4-1)graph_builder.add_edge(START, \"chatbot\") [](https://langchain-ai.github.io/langgraph/tutorials/introduction/#__codelineno-17-33)graph_builder.add_node(\"tools\", tool_node) [](https://langchain-ai.github.io/langgraph/tutorials/introduction/#__codelineno-18-19)# The `tools_condition` function returns \"tools\" if th

In [3]:
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Ya that's helpful. Maybe I'll "
                    "build an autonomous agent with it!"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Ya that's helpful. Maybe I'll build an autonomous agent with it!
================================== Ai Message ==================================

That sounds like an exciting project! Building an autonomous agent with LangGraph can allow you to explore various functionalities and create a system that interacts intelligently with users or other systems. Here are a few tips to help you get started:

1. **Define the Purpose**: Determine what you want your agent to accomplish. Whether it's answering questions, providing recommendations, or assisting with tasks, having a clear goal will guide your design.

2. **Utilize Multi-Tool Capabilities**: Leverage LangGraph's ability to integrate various tools. Think about what external services or APIs your agent might need to use.

3. **Handling Conversations**: Implement a strategy for managing conversation history, allowing your agent to maintain context over inter

In [4]:
# we can replay the full state history to see everything 
# that occurred.
to_replay = None
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
    if len(state.values["messages"]) == 6:
        # We are somewhat arbitrarily selecting a specific state based on the number of chat messages in the state.
        to_replay = state

Num Messages:  6 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  3 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  1 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  0 Next:  ('__start__',)
--------------------------------------------------------------------------------


In [5]:
# We've picked out to_replay as a state to resume from.
print(to_replay.next)
print(to_replay.config)

()
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1effa624-5b0d-60d6-8006-8d144d40fee2'}}


In [6]:
# Providing this checkpoint_id value tells LangGraph's 
# checkpointer to load the state from that moment in time.

# # The `checkpoint_id` in the `to_replay.config` corresponds 
# to a state we've persisted to our checkpointer.

for event in graph.stream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================

That sounds like an exciting project! Building an autonomous agent with LangGraph can allow you to explore various functionalities and create a system that interacts intelligently with users or other systems. Here are a few tips to help you get started:

1. **Define the Purpose**: Determine what you want your agent to accomplish. Whether it's answering questions, providing recommendations, or assisting with tasks, having a clear goal will guide your design.

2. **Utilize Multi-Tool Capabilities**: Leverage LangGraph's ability to integrate various tools. Think about what external services or APIs your agent might need to use.

3. **Handling Conversations**: Implement a strategy for managing conversation history, allowing your agent to maintain context over interactions. This ensures it can follow up on previous topics or continue discussions meaningfully.

4. **Experiment with Agent Architectures**: Explor